# Implementing Naive Bayes from scratch using an example

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
import matplotlib as mpl

In [7]:
mpl.rcParams["figure.figsize"] = (8,6)
mpl.rcParams["axes.grid"] = False

In [8]:
%matplotlib inline

In [9]:
X_train = np.array([
    [0,1,1],
    [0,0,1],
    [0,0,0],
    [1,1,0]
])
Y_train = ["Y", "N", "Y", "Y"]
X_test = np.array([[1,1,0]])